<a href="https://colab.research.google.com/github/tuanhdao/FYRE/blob/main/BVA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [12]:
# Open file and delete first 5 lines which we don't need to use and save it as a new clean file
with open('Acc_segments.txt') as f: #CHANGE THIS TO YOUR FILE'S NAME
    lines = f.readlines()
    del lines[0:5]
new_file = open("Acc_Segments_clean.txt", "w+")
for line in lines:
    clean_file = new_file.write(line)
new_file.close()
condition = input("Acc or Rej? This will be to name the file only. ")

Acc or Rej?Acc


In [13]:
#gets the number of lines
file = open("Acc_Segments_clean.txt", "r")
number_of_lines = 0
for line in file:
  number_of_lines += 1
print(number_of_lines)
file.close()

21572


In [ ]:
#import Pandas and display
import pandas as pd
from IPython.display import display

#Define a function to create sublists out of elements
def extract(lst):
    return [[e[-2:]] for e in lst]

#Get IDs
f = open('Acc_Segments_clean.txt')
count = 0  
#Okay this is kind of arbitrary, but I located the number of the line of each ID 
#Then created list with the starting line to the end line with 53 as the step 
#That is because each person has 53 lines of information
entry = range(2, number_of_lines, 53)  
list_id = []
for line in f:
  count += 1
  for n in entry:
    if count == n: 
      id = line
      id_clean = id.replace("\n","") #delete trailing white space
      list_id.append(id_clean)
id_df = pd.DataFrame (list_id,columns=['ID']) #create dataframe

#Next, create an input option for the electroders
lookup = input("Electrode (MUST BE WRITTEN AS IT IS IN THE MACRO): ")
number_segments = []
number_segments_all_other = []
number_segments_Cz_FCz = []
#Ok now the program will look up the line associated with the chosen electrode
for line in lines:
  if lookup in line:
    value = line.strip()
    value_clean = value.replace("\t","") #these 2 lines clean the data from trailing spaces and new lines
    if lookup == "Cz" or lookup == "FCz": #beware of Cz and FCz because they share the same characters in the name
      number_segments_Cz_FCz.append(value_clean) #separate list for Cz FCz values
      N=2
      sub_Cz_FCz_list = [number_segments_Cz_FCz[n:n+N] for n in range(0, 
      len(number_segments_Cz_FCz), N)] #this parses the list into smaller lists of 2 elements
      values_Cz_FCz = []
      for i in sub_Cz_FCz_list:
        for k in i:
          values_Cz_FCz.append(k[-2:]) #only take the last 2 values of each element in the list
      sub_Cz_FCz_list_2 = [values_Cz_FCz[n:n+N] for n in range(0, 
      len(values_Cz_FCz), N)] #pair the elements again to be in twos
      number_segments_df = pd.DataFrame(sub_Cz_FCz_list_2) #create a data frame
      frame = [id_df, number_segments_df]
    else:
      number_segments_all_other.append(value)  #for all other electrodes
      sub_other = extract(number_segments_all_other)
      number_segments_df = pd.DataFrame (sub_other)

#Create a list of IDs and segments
concatenate = [id_df, number_segments_df]
#merge ID and electrode values so that we add new variables, not cases
frame= pd.concat(concatenate, axis = 1) 
df = pd.DataFrame(frame)
final_renamed_df = df.copy(deep=True)

#rename columns to show which electrode is used
if lookup == "Cz" or lookup == "FCz":
  final_renamed_df.columns = ["ID", "Cz", "FCz"]
else:
  final_renamed_df.columns = ["ID", lookup]

#display the final data frame to check for accuracy)
display(final_renamed_df)

#Save to a CSV file
final_renamed_df.to_csv("/content/drive/MyDrive/" + "{}{}{}.csv".format(condition, "_", lookup), index = False)
